## RAG implementation
https://huggingface.co/docs/datasets/how_to_metrics

### Preprocess data

In [1]:
import pandas as pd
from tqdm import tqdm

def generate_UVQuAD(df: pd.DataFrame, output_path: str = "UVQuAD.csv"):
    """Generate UVQuAD from data_raw.csv"""
    new_data = []
    for i in tqdm(range(len(df))):
        request = eval(df["pergunta"][i])
        for p in request["perguntas"]:
            # print(request["perguntas"][p])
            # print(type(request["respostas"][p]))
            new_data.append(
                {
                    "title": df["title"][i],
                    "context": df["context"][i],
                    "question": request["perguntas"][p],
                    "answer": {
                        "text": list(request["respostas"][p].values()),
                        "answer_start": [],},
                }
            )
    new_df = pd.DataFrame(new_data)
    new_df.to_csv(output_path, index=False)
    return new_df

# Load data_raw
df = pd.read_csv('../data/processed/data_raw.csv')

# Generate UVQuAD
df = generate_UVQuAD(df, "../data/processed/UVQuAD.csv")

100%|██████████| 422/422 [00:00<00:00, 7472.37it/s]


### Retriever Probe

In [2]:
!pip install rank_bm25

In [3]:
from langchain.retrievers import BM25Retriever


In [8]:
df = pd.read_csv('../data/processed/contextos.csv')
df.head(1)

,title,context
0,titulo,Vestibular Unicamp 2024\nResolução GR-031/2023...


In [9]:
retriever = BM25Retriever.from_texts(df['context'].to_list())

In [10]:
result = retriever.get_relevant_documents("Quantas vagas regulares são oferecidas para ingresso nos Cursos de Graduação da Unicamp em 2024?")
result

[Document(page_content='Art. 1º Para o ano de 2024 são oferecidas 3340 vagas regulares para ingresso nos Cursos de Graduação da Unicamp distribuídas nos seguintes sistemas de ingresso:\xa0 I. 2537 vagas oferecidas pelo Vestibular Unicamp (VU) 2024.\xa0 II. 314 vagas oferecidas pelo Edital ENEM-Unicamp 2024.\xa0 III. 325 vagas oferecidas pelo Provão Paulista 2024.\xa0 IV. 49 vagas oferecidas pelo Vestibular Indígena (VI) 2024. O Vestibular Indígena terá ainda 81 vagas adicionais, conforme Edital a ser publicado, respeitando os princípios da Deliberação CONSU-A-032/2017.\xa0 IV. 115 vagas oferecidas pelo Edital de olimpíadas cientíﬁcas e competições de conhecimento de áreas especíﬁcas. Haverá, ainda, 14 vagas adicionais nesse sistema de ingresso, conforme Edital a ser publicado, respeitando os princípios da Deliberação CONSU-A-032/2017.\xa0'),
 Document(page_content='§1º As vagas regulares não preenchidas nos incisos II, III, IV e V serão transferidas para o VU 2024 para que sejam preenc

### Retriever Class

In [12]:
from langchain.retrievers import BM25Retriever
from openai import OpenAI
import pandas as pd
import os

class ConvHistory:
    def __init__(self):
        self.messages = []
        self.offset = 0
        self.system_message = "¡Hola! Soy tu chatbot para el Vestibular da Unicamp 2024. ¿En qué puedo ayudarte?"
        
    def set_system_message(self, system_message: str):
        """Set the system message."""
        self.system_message = system_message

    def append_message(self, role: str, message: str):
        """Append a new message."""
        self.messages.append([role, message])
    
    def to_openai_api_messages(self):
        """Convert the conversation to OpenAI chat completion format."""
        ret = [{"role": "system", "content": self.system_message}]

        for i, (_, msg) in enumerate(self.messages[self.offset :]):
            if i % 2 == 0:
                ret.append({"role": "user", "content": msg})
            else:
                if msg is not None:
                    ret.append({"role": "assistant", "content": msg})
        return ret

class UVQuADChatBotRetriever:
    def __init__(self, data_path: str = "../data/processed/context_data.csv"):
        self.data = pd.read_csv(data_path)
        self.from_texts(self.data['context'].to_list())
        
    def from_texts(self, texts: list):
        self.retriever = BM25Retriever.from_texts(texts)
        
    def get_relevant_documents(self, question: str, n_docs: int = 1):
        result = self.retriever.get_relevant_documents(question)
        return result

class GPT3ChatBot:
    def __init__(self, model: str = "gpt-3.5-turbo",
                api_key: str = os.getenv("OPENAI_API_KEY"),
                retriever_data_path: str = "../data/processed/context_data.csv"):
        self.model = model
        self.agent = OpenAI(
            api_key= api_key,
        )
        self.retriever = UVQuADChatBotRetriever(retriever_data_path)
        self.conv = ConvHistory()
        self.conv.set_system_message("Vocé é um chatbot para o Vestibular da Unicamp 2024.")
        
    def retriever_set_message(self, question: str):
        context = self.retriever_get_context(question)
        user_message = f"Com base no contexto: {context}, sesponda a pergunta: {question}.Resposta:"
        self.conv.append_message("user", user_message)
        # return context['context']
    
    def retriever_get_context(self, question: str):
        relevant_documents = self.retriever.get_relevant_documents(question)
        return relevant_documents[0]
    
    def get_answer_from_conversation(self, conversation: list):
        return self.get_answer(conversation)
    
    def get_answer_from_gpt3(self, question: str):
        self.retriever_set_message(question)
        response = self.agent.chat.completions.create(
            model=self.model,
            messages=self.conv.to_openai_api_messages(),
            max_tokens=100,
        )
        self.conv.append_message("assistant", response.choices[0].message.content)
        return response.choices[0].message.content


In [36]:
q = "Quantas vagas regulares são oferecidas para ingresso nos Cursos de Graduação da Unicamp em 2024?"
retriever = UVQuADChatBotRetriever()
retriever.get_relevant_documents(q)

[Document(page_content='Art. 1º Para o ano de 2024 são oferecidas 3340 vagas regulares para ingresso nos Cursos de Graduação da Unicamp distribuídas nos seguintes sistemas de ingresso:\xa0 I. 2537 vagas oferecidas pelo Vestibular Unicamp (VU) 2024.\xa0 II. 314 vagas oferecidas pelo Edital ENEM-Unicamp 2024.\xa0 III. 325 vagas oferecidas pelo Provão Paulista 2024.\xa0 IV. 49 vagas oferecidas pelo Vestibular Indígena (VI) 2024. O Vestibular Indígena terá ainda 81 vagas adicionais, conforme Edital a ser publicado, respeitando os princípios da Deliberação CONSU-A-032/2017.\xa0 IV. 115 vagas oferecidas pelo Edital de olimpíadas cientíﬁcas e competições de conhecimento de áreas especíﬁcas. Haverá, ainda, 14 vagas adicionais nesse sistema de ingresso, conforme Edital a ser publicado, respeitando os princípios da Deliberação CONSU-A-032/2017.\xa0'),
 Document(page_content='§1º As vagas regulares não preenchidas nos incisos II, III, IV e V serão transferidas para o VU 2024 para que sejam preenc

In [37]:
chatbot = GPT3ChatBot()
chatbot.conv.offset = -4
chatbot.get_answer_from_gpt3(q)

'Para o ano de 2024, são oferecidas 3340 vagas regulares para ingresso nos Cursos de Graduação da Unicamp.'

In [38]:
chatbot.conv.messages

[['user',
  "Com base no contexto: page_content='Art. 1º Para o ano de 2024 são oferecidas 3340 vagas regulares para ingresso nos Cursos de Graduação da Unicamp distribuídas nos seguintes sistemas de ingresso:\\xa0 I. 2537 vagas oferecidas pelo Vestibular Unicamp (VU) 2024.\\xa0 II. 314 vagas oferecidas pelo Edital ENEM-Unicamp 2024.\\xa0 III. 325 vagas oferecidas pelo Provão Paulista 2024.\\xa0 IV. 49 vagas oferecidas pelo Vestibular Indígena (VI) 2024. O Vestibular Indígena terá ainda 81 vagas adicionais, conforme Edital a ser publicado, respeitando os princípios da Deliberação CONSU-A-032/2017.\\xa0 IV. 115 vagas oferecidas pelo Edital de olimpíadas cientíﬁcas e competições de conhecimento de áreas especíﬁcas. Haverá, ainda, 14 vagas adicionais nesse sistema de ingresso, conforme Edital a ser publicado, respeitando os princípios da Deliberação CONSU-A-032/2017.\\xa0', sesponda a pergunta: Quantas vagas regulares são oferecidas para ingresso nos Cursos de Graduação da Unicamp em 2024

In [42]:
q = "Quais são os sistemas de ingresso para os cursos de graduação da Unicamp em 2024?"
chatbot.get_answer_from_gpt3(q)

'Os sistemas de ingresso para os cursos de graduação da Unicamp em 2024 são:\n\nI. Vestibular Unicamp (VU) 2024\nII. Edital ENEM-Unicamp 2024\nIII. Vestibular Indígena (VI) 2024\nIV. Edital de olimpíadas científicas e competições de conhecimento de áreas específicas.\n\nEsses são os sistemas de ingresso menc'

In [43]:
chatbot.conv.messages[-4:]

[['user',
  "Com base no contexto: page_content='Vestibular Unicamp 2024\\nResolução GR-031/2023, de 13/07/2023\\nReitor: Maria Luiza Moretti - Reitora em exercício\\nO Reitor da Universidade Estadual de Campinas, considerando a Deliberação CONSU-A-032/2017 de 21 de novembro de 2017, que especiﬁca sobre os sistemas de ingresso aos Cursos de Graduação da Unicamp, torna pública a Resolução Vestibular Unicamp 2024 para vagas no ensino de Graduação.', sesponda a pergunta: Quais são os sistemas de ingresso para os cursos de graduação da Unicamp em 2024?.Resposta:"],
 ['assistant',
  'Os sistemas de ingresso para os cursos de graduação da Unicamp em 2024 são:\n\nI. Vestibular Unicamp (VU) 2024\nII. Edital ENEM-Unicamp 2024\nIII. Vestibular Indígena (VI) 2024\nIV. Edital de olimpíadas científicas e competições de conhecimento de áreas específicas.\n\nNo entanto, não há menção ao'],
 ['user',
  "Com base no contexto: page_content='Vestibular Unicamp 2024\\nResolução GR-031/2023, de 13/07/2023\

Dataset de Predicciones

In [51]:
import pandas as pd
from tqdm import tqdm

# Load UVQuAD
UVQuAD = pd.read_csv('../data/processed/UVQuAD.csv')

chatbot = GPT3ChatBot(
    retriever_data_path="../data/processed/context_data.csv"
)
chatbot.conv.offset = -4
data = []
for i in tqdm(UVQuAD.index[:4]):
    q = UVQuAD['question'][i]
    response = chatbot.get_answer_from_gpt3(q)
    data.append(
        {
            'id': i,
            "prediction_text": response,
        }
    )
df = pd.DataFrame(data)
df

  0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 4/4 [00:06<00:00,  1.75s/it]


,id,prediction_text
0,0,A Resolução Vestibular Unicamp 2024 foi public...
1,1,O Reitor da Universidade Estadual de Campinas ...
2,2,A finalidade da Resolução Vestibular Unicamp 2...
3,3,"No vestibular Unicamp 2024, são oferecidas 253..."


In [52]:
df.to_dict(orient='records')

[{'id': 0,
  'prediction_text': 'A Resolução Vestibular Unicamp 2024 foi publicada no dia 13 de julho de 2023.'},
 {'id': 1,
  'prediction_text': 'O Reitor da Universidade Estadual de Campinas é Maria Luiza Moretti, que está exercendo o cargo atualmente.'},
 {'id': 2,
  'prediction_text': 'A finalidade da Resolução Vestibular Unicamp 2024 é tornar pública a regulamentação e diretrizes para o processo seletivo de ingresso nos cursos de graduação da Universidade Estadual de Campinas (Unicamp) para o ano de 2024. Ela estabelece as regras, critérios e procedimentos que serão utilizados no vestibular.'},
 {'id': 3,
  'prediction_text': 'No vestibular Unicamp 2024, são oferecidas 2537 vagas regulares para ingresso nos cursos de graduação da Universidade Estadual de Campinas (Unicamp).'}]

In [50]:
UVQuAD = pd.read_csv('../data/processed/UVQuAD.csv')
UVQuAD["answer"] = UVQuAD["answer"].apply(lambda x: eval(x))
type(UVQuAD["answer"][0])

dict